In [ ]:
# Memory Biased Random Walk with Segment Analysis
# For Identifying Community-Like Structures in Networks
# This is also called Network Clustering

# For further information: Yucel et al., 2016, Detection of network communities with memory-biased random walk algorithms,
#                                              Journal of Complex Networks
# M. Yucel, Unpublished PhD Thesis, Memory Biased Random Walk As An Empirical Approach to Indetifying Network Communities

# Author: Dr. Mesut Yucel - 2016

# In this implementation, an Object Oriented Design is used for simulating MBRW.

In [ ]:
import pandas as pd
import numpy as np
import itertools
from collections import deque
from operator import itemgetter
from scipy.special import comb
from functools import reduce

In [ ]:
# Defining each node as an object with data members of ID and neighbors...

class Node:
    
    ' The general node class for network nodes. '
    
    def __init__(self, ID, Neighbors):
        
        self.ID = ID
        self.Neighbors = Neighbors
        
        
    def get_id(self):
        
        return self.ID
    
    
    def get_neighbors(self):
        
        return self.Neighbors
    

In [ ]:
# Read binary interaction data as a Binary Interaction Matrix

bin_int_mat = pd.read_csv(r'./desktop/bin_int_mat.csv', dtype=np.uint16)
N = np.max(np.max(bin_int_mat)) + 1

# Convert the Binary Interaction Matrix into a list of node objects with ID and neighbors..

neighbor_objects = []

for node_id in range(N):
    
    neighbor_indices = [bin_int_mat[bin_int_mat[Node] == node_id].index.tolist() for Node in bin_int_mat.columns]
    neighbor_indices = list(itertools.chain.from_iterable(neighbor_indices))
    
    neighbors = np.append(bin_int_mat.iloc[neighbor_indices]['Node1'], bin_int_mat.iloc[neighbor_indices]['Node2'])
    neighbors = np.setdiff1d(neighbors, node_id)
    
    neighbor_objects.append(Node(node_id, neighbors))
    
    
del bin_int_mat

In [ ]:
# Simulation Parameters

alpha = 1000
memory_size = 5
circulation_number = 2

current_node = np.random.randint(N)
previous_node = N
memorized_sequence = N * np.ones(memory_size)
memorized_sequence[-1] = current_node
visited_nodes = np.zeros(N)
visited_nodes[current_node] = 1
traversed_sequence = deque([current_node])

circulation_time = 0

In [ ]:
# Memory Biased Random Walk Simulation

while circulation_time < circulation_number:
    
    potential_next_nodes = (neighbor_objects[current_node]).get_neighbors()
    potential_next_nodes = np.setdiff1d(potential_next_nodes, previous_node)     # No reverse back...
    
    biases = np.ones(len(potential_next_nodes))
    
    
    if current_node in memorized_sequence[:-1]:
        
        biased_indices = np.where(memorized_sequence[:-1] == current_node)
        biased_indices = biased_indices[0] + 1
        biased_nodes = memorized_sequence[biased_indices]
        
        for node in biased_nodes:
            biases[np.where(potential_next_nodes == node)] = alpha
    
    
    
    # Choosing the next node with Fitness Proportionate Selection
    
    random_selection = biases.sum() * np.random.rand()
    previous_node = current_node
    current_node = potential_next_nodes[np.argmax(biases.cumsum() > random_selection)]
    
    visited_nodes[current_node] = 1
    traversed_sequence.append(current_node)
    
    
    if visited_nodes.all():
        circulation_time += 1
        visited_nodes[:] = 0
    
    
    memorized_sequence[:-1] = memorized_sequence[1:]
    memorized_sequence[-1] = current_node
    

sequence_array = np.array(traversed_sequence)

del traversed_sequence

print('Simulation has been completed...')

In [ ]:
# Function definiton for calculating Module Density

def module_density(array):
    
    if len(array) > 1:
        max_link_number = comb(len(array), 2)
        intra_link_number = 0.0
        total_link_number = 0.0
        for node in array:
            intra_link_number += (np.in1d((neighbor_objects[node]).get_neighbors(), array)).sum()
            total_link_number += len((neighbor_objects[node]).get_neighbors())
        intra_link_number /= 2
        relative_connectivity = intra_link_number / max_link_number
        intra_connection_ratio = 2 * intra_link_number / total_link_number
    
        return relative_connectivity * intra_connection_ratio
    else:
        return 0

In [ ]:
# Modularity Analysis

# Stage 1: Overlapping Unique Candidate Communities

window_size = N
nonunique_modules = [np.unique(sequence_array[start:start + window_size]) 
                       for start in range(0, len(sequence_array), window_size)]

nonunique_modules = np.array(sorted(nonunique_modules, key=itemgetter(0,1,2)))

unique_codes = np.array([True] * len(nonunique_modules))

for module_ind1 in range(len(nonunique_modules)-1):
    
    if unique_codes[module_ind1]:
        module_1 = nonunique_modules[module_ind1]
        module_ind2 = module_ind1 + 1
        module_2 = nonunique_modules[module_ind2]
        
        while np.array_equal(module_1[:3], module_2[:3]) and module_ind2 < len(nonunique_modules) - 1:
            
            if np.array_equal(module_1, module_2):
                unique_codes[module_ind2] = False
                
            module_ind2 += 1
            module_2 = nonunique_modules[module_ind2]
            
        
unique_modules = nonunique_modules[unique_codes]
unique_modules = [(0, module) for module in unique_modules]

del nonunique_modules

In [ ]:
# Stage 2: Non-Overlapping Unique Communities

nonoverlapping_modules = []
temp_module = np.array([])


while True:
    
    unique_modules = [(module_density(np.setdiff1d(module[1], temp_module)), np.setdiff1d(module[1], temp_module)) 
                      for module in unique_modules if len(np.setdiff1d(module[1], temp_module)) > 2]
   
    if unique_modules != []:
        
        unique_modules = sorted(unique_modules, key=itemgetter(0), reverse=True)
        nonoverlapping_modules.append(unique_modules[0])
        temp_module = unique_modules[0][1]
        del unique_modules[0]
    else:
        break


module_list = [module[1] for module in nonoverlapping_modules]
initial_gate_nodes = np.setdiff1d(np.arange(N), reduce(np.union1d, module_list))

del nonoverlapping_modules

In [ ]:
# Stage 3: Processing Gate Nodes

gate_nodes = []

for node in initial_gate_nodes:
    
    gate_node_neighbors = (neighbor_objects[node]).get_neighbors()
    link_distribution = [len(np.intersect1d(gate_node_neighbors, module, assume_unique=True)) for module in module_list]
    linked_module_indices = np.argwhere(link_distribution == np.amax(link_distribution))
    
    if len(linked_module_indices) == 1:
        
        module_list[linked_module_indices[0][0]] = sorted(np.append(module_list[linked_module_indices[0][0]], node))
        
    else:
        gate_nodes.append(node)

        
module_list = [(module_density(module), module) for module in module_list]
module_list = sorted(module_list, key=itemgetter(0), reverse=True)

In [ ]:
# Organizing the Results in Module Dictionary

modules = {}

for module_id, module in enumerate(module_list):
    modules[module_id] = {'ModuleDensity': module[0], 'ModuleSize': len(module[1]), 'ModuleNodes': module[1]}


# Adding the gate nodes as a separate group

modules['GateNodes'] = {'ModuleDensity': module_density(gate_nodes), 'ModuleSize': len(gate_nodes), 'ModuleNodes': gate_nodes}

In [ ]:
# Summary Results

print('\n\nSummary of Network Community Statistics\n---------------------------------------\n')
print('Number of Modules Identified: ', len(modules) - 1, 'Modules', ' + ', modules['GateNodes']['ModuleSize'], 'Gate Nodes')
print('Average Module Density: ', np.array([modules[i]['ModuleDensity'] for i in range(len(modules) - 1)]).mean())
print('Average Module Size: ', np.array([modules[i]['ModuleSize'] for i in range(len(modules) - 1)]).mean(), 'Nodes\n')

In [ ]:
# Graphical Output

import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
module_densities = [module['ModuleDensity'] for module in modules.values()]
plt.figure(figsize=(14,8))
plt.plot(range(1, len(modules)+1), module_densities, 'o-', lw=2)
plt.xlabel('Module Rank (From Highest Module Density to Lowest)')
plt.ylabel('Module Density')
plt.xlim([0, len(modules)+1])
mpl.rc('font', size=16)
plt.grid(True)